In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling

In [ ]:
df_pings = pd.read_csv('./../data/pings.csv')

In [ ]:
## Count of Unique drivers in Pings.csv
len(np.unique(df_pings.driver_id.values))

2480

In [ ]:
## Processed Driver Hours Data
driver_hours = pd.read_csv('../processed_data_and_models/Pings_Grouped_Data.csv')

In [ ]:
## VERIFIED | Data rows: Unique Driver Ids * 21days = 2480 * 21 = 52080
assert 52080 == len(driver_hours)

In [ ]:
driver_profile = pd.read_csv('../processed_data_and_models/driver.csv')

In [ ]:
driver_profile.head(10)

,driver_id,gender,age,number_of_kids
0,979863,MALE,26,2
1,780123,MALE,60,2
2,614848,MALE,45,4
3,775046,MALE,62,3
4,991601,MALE,23,0
5,670477,MALE,36,1
6,194053,MALE,35,0
7,364596,FEMALE,36,4
8,640428,MALE,28,0
9,154517,MALE,25,2


In [ ]:
## Driver_Id for which we don't have PingData or they were not available 
set(np.unique(driver_profile.driver_id.values)) - set(np.unique(driver_hours.driver_id.values))

{230923,
 373792,
 425331,
 523243,
 585955,
 616243,
 675613,
 682678,
 743899,
 756684,
 772057,
 808404,
 854976,
 934994,
 971478,
 993757,
 998740}

### Merge Data | Driver Profile and Driver Hours on = [driver_id]

In [ ]:
## Left Join
dataset = pd.merge(driver_profile, driver_hours, on = ['driver_id'], how = 'left')

In [ ]:
dataset[dataset.driver_id == 230923]

,driver_id,gender,age,number_of_kids,date,available_hours,dayofweek
17518,230923,MALE,18,0,NaN,NaN,NaN


In [ ]:
dataset['date'].fillna('2017-06-01',inplace = True)

In [ ]:
## Fill in missing Date For Driver with Available_hours = 0
dataset.date = pd.to_datetime(dataset.date)
dataset = dataset.set_index(
    ['date', 'driver_id']
).unstack().fillna(method = 'ffill').asfreq(
    'D'
).stack().sort_index(level=1).reset_index()

In [ ]:
dataset['dayofweek'].fillna(dataset['date'].dt.dayofweek, inplace = True)
dataset['available_hours'].fillna(0,inplace = True)

In [ ]:
## Verifying Data of Driver which was missing
dataset[dataset.driver_id == 998740]

,date,driver_id,gender,age,number_of_kids,available_hours,dayofweek
52416,2017-06-01,998740,MALE,27.0,0.0,0.0,3.0
52417,2017-06-02,998740,MALE,27.0,0.0,0.0,4.0
52418,2017-06-03,998740,MALE,27.0,0.0,0.0,5.0
52419,2017-06-04,998740,MALE,27.0,0.0,0.0,6.0
52420,2017-06-05,998740,MALE,27.0,0.0,0.0,0.0
52421,2017-06-06,998740,MALE,27.0,0.0,0.0,1.0
52422,2017-06-07,998740,MALE,27.0,0.0,0.0,2.0
52423,2017-06-08,998740,MALE,27.0,0.0,0.0,3.0
52424,2017-06-09,998740,MALE,27.0,0.0,0.0,4.0
52425,2017-06-10,998740,MALE,27.0,0.0,0.0,5.0


#### Dataset len = Number of Drivers * 21 days => 2497*21 => 52437

In [ ]:
assert len(dataset) == 52437

### Label Encoding: Replace Male: 0 and Female: 1

In [ ]:
dataset['gender'].replace({'FEMALE':1, 'MALE':0}, inplace = True)

### Encoding-> 1: Weekend  0: Weekday

In [ ]:
dataset['weekend'] = dataset['dayofweek'].apply(lambda x: 0 if x < 5 else 1)

### Training Data is Ready

#### Data Analysis of Training Data

In [ ]:
dataset.to_csv('../processed_data_and_models/Training_Dataset.csv',index = False)

In [ ]:
profile = dataset.profile_report(title='Driver Training Data')

In [ ]:
profile.to_file('../data_analysis_EDA/Driver_Training.html')

### Preparing Test Data

In [ ]:
test_data = pd.read_csv('./../data/test.csv')

In [ ]:
test_data['date'] = pd.to_datetime(test_data['date']) 
test_data['dayofweek'] = test_data['date'].dt.dayofweek
test_data['weekend'] = test_data['dayofweek'].apply(lambda x: 0 if x < 5 else 1)

In [ ]:
test = pd.merge(test_data,driver_profile, on = ['driver_id'])

In [ ]:
test['gender'].replace({'FEMALE':1, 'MALE':0}, inplace = True)
test.to_csv('../processed_data_and_models/driver_test.csv',index = False)